In [1]:
import pandas as pd
import os

In [2]:
%pwd

'/home/omar/Desktop/End-to-end-Machine-Learning-Project-with-MLflow/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/omar/Desktop/End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
df = pd.read_csv('artifacts/data_ingestion/predictive_maintenance.csv')
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Target                   10000 non-null  int64  
 9   Failure Type             10000 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 781.4+ KB


In [7]:
df.rename(columns={'Target': 'Machine failure'}, inplace=True)

In [8]:
df.drop(['UDI', 'Product ID'], axis=1, inplace=True)
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Failure Type
0,M,298.1,308.6,1551,42.8,0,0,No Failure
1,L,298.2,308.7,1408,46.3,3,0,No Failure
2,L,298.1,308.5,1498,49.4,5,0,No Failure
3,L,298.2,308.6,1433,39.5,7,0,No Failure
4,L,298.2,308.7,1408,40.0,9,0,No Failure


kelvin to Celsius

In [9]:
df['Air temperature [c]'] = df['Air temperature [K]'] - 273.15
df['Process temperature [c]'] = df['Process temperature [K]'] - 273.15
df.drop(['Air temperature [K]', 'Process temperature [K]'], axis=1, inplace=True)
df.head()

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Failure Type,Air temperature [c],Process temperature [c]
0,M,1551,42.8,0,0,No Failure,24.95,35.45
1,L,1408,46.3,3,0,No Failure,25.05,35.55
2,L,1498,49.4,5,0,No Failure,24.95,35.35
3,L,1433,39.5,7,0,No Failure,25.05,35.45
4,L,1408,40.0,9,0,No Failure,25.05,35.55


In [10]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['L', 'M', 'H']])

df['Type'] = encoder.fit_transform(df[['Type']])
df.head()

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Failure Type,Air temperature [c],Process temperature [c]
0,1.0,1551,42.8,0,0,No Failure,24.95,35.45
1,0.0,1408,46.3,3,0,No Failure,25.05,35.55
2,0.0,1498,49.4,5,0,No Failure,24.95,35.35
3,0.0,1433,39.5,7,0,No Failure,25.05,35.45
4,0.0,1408,40.0,9,0,No Failure,25.05,35.55


In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df['type_of_failure'] = encoder.fit_transform(df['Failure Type'])
df.head()

,Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Failure Type,Air temperature [c],Process temperature [c],type_of_failure
0,1.0,1551,42.8,0,0,No Failure,24.95,35.45,1
1,0.0,1408,46.3,3,0,No Failure,25.05,35.55,1
2,0.0,1498,49.4,5,0,No Failure,24.95,35.35,1
3,0.0,1433,39.5,7,0,No Failure,25.05,35.45,1
4,0.0,1408,40.0,9,0,No Failure,25.05,35.55,1


In [12]:
encoder.classes_

array(['Heat Dissipation Failure', 'No Failure', 'Overstrain Failure',
       'Power Failure', 'Random Failures', 'Tool Wear Failure'],
      dtype=object)

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

col_to_scale = ['Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Air temperature [c]', 'Process temperature [c]']

df_scaled = scaler.fit_transform(df[col_to_scale])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = col_to_scale

df.drop(col_to_scale, axis=1, inplace=True)

df_scaled = pd.concat([df, df_scaled], axis=1)
df_scaled.head()

,Type,Machine failure,Failure Type,type_of_failure,Rotational speed [rpm],Torque [Nm],Tool wear [min],Air temperature [c],Process temperature [c]
0,1.0,0,No Failure,1,0.222934,0.535714,0.000000,0.304348,0.358025
1,0.0,0,No Failure,1,0.139697,0.583791,0.011858,0.315217,0.370370
2,0.0,0,No Failure,1,0.192084,0.626374,0.019763,0.304348,0.345679
3,0.0,0,No Failure,1,0.154249,0.490385,0.027668,0.315217,0.358025
4,0.0,0,No Failure,1,0.139697,0.497253,0.035573,0.315217,0.370370


In [16]:
X = df_scaled.drop('type_of_failure', axis=1)
y = df_scaled['type_of_failure']

In [17]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy='auto')

X_resampled, y_resampled = ros.fit_resample(X, y)

df_sampled = pd.concat([X_resampled, y_resampled], axis=1)


   Type  Machine failure Failure Type  Rotational speed [rpm]  Torque [Nm]  \
0   1.0                0   No Failure                0.222934     0.535714   
1   0.0                0   No Failure                0.139697     0.583791   
2   0.0                0   No Failure                0.192084     0.626374   
3   0.0                0   No Failure                0.154249     0.490385   
4   0.0                0   No Failure                0.139697     0.497253   

   Tool wear [min]  Air temperature [c]  Process temperature [c]  \
0         0.000000             0.304348                 0.358025   
1         0.011858             0.315217                 0.370370   
2         0.019763             0.304348                 0.345679   
3         0.027668             0.315217                 0.358025   
4         0.035573             0.315217                 0.370370   

   type_of_failure  
0                1  
1                1  
2                1  
3                1  
4                

In [18]:
df_sampled.head()


,Type,Machine failure,Failure Type,Rotational speed [rpm],Torque [Nm],Tool wear [min],Air temperature [c],Process temperature [c],type_of_failure
0,1.0,0,No Failure,0.222934,0.535714,0.000000,0.304348,0.358025,1
1,0.0,0,No Failure,0.139697,0.583791,0.011858,0.315217,0.370370,1
2,0.0,0,No Failure,0.192084,0.626374,0.019763,0.304348,0.345679,1
3,0.0,0,No Failure,0.154249,0.490385,0.027668,0.315217,0.358025,1
4,0.0,0,No Failure,0.139697,0.497253,0.035573,0.315217,0.370370,1


In [19]:
df_sampled.value_counts('type_of_failure')

type_of_failure
0    9652
1    9652
2    9652
3    9652
4    9652
5    9652
Name: count, dtype: int64

In [ ]:
#df_sampled.to_csv('../data/processed/data_processed.csv', index=False)